In [38]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import networkx as nx
import osmnx as ox
# from haversine import haversine, Unit

import numpy as np


In [39]:
df0 = pd.read_csv('trip_df_block_agg1.csv')
df0 = df0.drop(['Unnamed: 0'], axis=1)

df0.columns

Index(['blockF', 'blockT', 'Trips_median', 'Trips_mean', 'Trips_sum', 'dist',
       'x_f', 'y_f', 'x_t', 'y_t', 'placeF', 'placeT'],
      dtype='object')

In [40]:
from pyproj import Proj, transform


# Define the UTM projection for Zone 10N
utm_proj = Proj(proj='utm', zone=10, ellps='WGS84', south=False)  # north=True is the default, explicitly south=False
wgs84_proj = Proj(proj='latlong', ellps='WGS84')

# Function to convert coordinates
def convert_utm_to_latlong(x, y):
    longitude, latitude = transform(utm_proj, wgs84_proj, x, y)
    return latitude, longitude

# Apply the conversion function
df0['Lat_from'], df0['Lon_from'] = zip(*df0.apply(lambda row: convert_utm_to_latlong(row['x_f'], row['y_f']), axis=1))
df0['Lat_to'], df0['Lon_to'] = zip(*df0.apply(lambda row: convert_utm_to_latlong(row['x_t'], row['y_t']), axis=1))


C:\Users\behnamt\AppData\Local\Temp\ipykernel_19092\1590544308.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude, latitude = transform(utm_proj, wgs84_proj, x, y)
C:\Users\behnamt\AppData\Local\Temp\ipykernel_19092\1590544308.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude, latitude = transform(utm_proj, wgs84_proj, x, y)


In [41]:
# print(len(df00))
print(len(df0))

2843


In [42]:
df0.head(4)

,blockF,blockT,Trips_median,Trips_mean,Trips_sum,dist,x_f,y_f,x_t,y_t,placeF,placeT,Lat_from,Lon_from,Lat_to,Lon_to
0,1,2.0,80.0,79.384615,3096,34902.883328,599623.625084,4.398832e+06,624097.626033,4.374564e+06,Chico,Oroville,39.733562,-121.837417,39.511753,-121.556438
1,1,4.0,11.0,14.897436,581,37814.183100,599481.421163,4.399995e+06,611584.801517,4.363708e+06,Chico,Biggs,39.744053,-121.838900,39.415680,-121.703772
2,1,5.0,55.0,58.615385,2286,44318.311069,599093.854914,4.400183e+06,612346.113732,4.357752e+06,Chico,Gridley,39.745790,-121.843395,39.361928,-121.695930
3,1,6.0,5.0,7.270270,269,60450.198899,601596.397441,4.397622e+06,643825.053506,4.356865e+06,Chico,Bangor,39.722431,-121.814586,39.349266,-121.330854


In [43]:
# read driving network
G = ox.graph_from_place("Butte county, CA, USA ", network_type="drive")
G = ox.add_edge_speeds(G) 
G = ox.add_edge_travel_times(G)

In [44]:
G.nodes

NodeView((86370002, 86370007, 86370011, 86370018, 86370021, 86370197, 86370214, 86370244, 86370254, 86370259, 86370265, 86370297, 86370308, 86370320, 86370324, 86370344, 86370367, 86370388, 86370487, 86370488, 86370490, 86370492, 86370493, 86370497, 86370499, 86370508, 86370517, 86370748, 86370756, 86370760, 86370777, 86370786, 86370789, 86370791, 86370795, 86370797, 86370803, 86370807, 86370812, 86370814, 86370883, 86370949, 86370958, 86371470, 86371474, 86371548, 86371598, 86371892, 86371894, 86371902, 86371907, 86371912, 86371922, 86371934, 86372030, 86372150, 86372455, 86372461, 86372618, 86373021, 86373041, 86373043, 86373044, 86373058, 86373144, 86373189, 86373194, 86373213, 86373244, 86373253, 86373257, 86373263, 86373264, 86373795, 86373815, 86374781, 86374820, 86374826, 86374827, 86374835, 86375432, 86375435, 86375439, 86375543, 86375792, 86375796, 86375800, 86375804, 86375808, 86375812, 86375929, 86376105, 86376129, 86376154, 86376163, 86376200, 86376488, 86376494, 86376577, 

In [54]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

# Assuming 'G' is already loaded as per your code

def drivingCost(df, G, ori_x_col, ori_y_col, dest_x_col, dest_y_col):
    df['network_node_id_ori'] = np.nan
    df['network_node_id_dest'] = np.nan
    df['driving_dist'] = np.nan
    df['driving_time'] = np.nan

    i = 0
    for index, row in df.iterrows():
        i =  i+1
        print(i)
        # Obtain the nearest network node ids for origin and destination
        network_node_id_ori = ox.distance.nearest_nodes(G, row[ori_x_col], row[ori_y_col])
        network_node_id_dest = ox.distance.nearest_nodes(G, row[dest_x_col], row[dest_y_col])
        
        # Store the node ids back to the DataFrame
        df.at[index, 'network_node_id_ori'] = network_node_id_ori
        df.at[index, 'network_node_id_dest'] = network_node_id_dest

        # Calculate driving distance and time using the shortest path Dijkstra's algorithm
        try:
            driving_dist = nx.dijkstra_path_length(G, source=network_node_id_ori, target=network_node_id_dest, weight='length')
            driving_time = nx.dijkstra_path_length(G, source=network_node_id_ori, target=network_node_id_dest, weight='travel_time')/60
        except nx.NetworkXNoPath:
            driving_dist = np.nan
            driving_time = np.nan

        # Store the calculated values back to the DataFrame
        df.at[index, 'driving_dist'] = driving_dist
        df.at[index, 'driving_time'] = driving_time

    return df


In [46]:
df0.columns

Index(['blockF', 'blockT', 'Trips_median', 'Trips_mean', 'Trips_sum', 'dist',
       'x_f', 'y_f', 'x_t', 'y_t', 'placeF', 'placeT', 'Lat_from', 'Lon_from',
       'Lat_to', 'Lon_to'],
      dtype='object')

In [48]:
df1 = drivingCost(df0, G,'Lon_from', 'Lat_from', 'Lon_to', 'Lat_to')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [59]:
# Load data
stops = pd.read_csv(r'.\google_transit\stops.txt')
trips = pd.read_csv(r'.\google_transit\trips.txt')
routes = pd.read_csv(r'.\google_transit\routes.txt')
stop_times = pd.read_csv(r'.\google_transit\stop_times.txt')


In [61]:
import pandas as pd
import numpy as np

# Constants
WALKING_SPEED = 1.4  # meters per second, typical walking speed


def calculate_distance(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance between two points in meters."""
    R = 6371000  # radius of Earth in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

def find_nearest_stop(lat, lon, stops):
    """Find the nearest stop to a given latitude and longitude."""
    distances = stops.apply(lambda stop: calculate_distance(lat, lon, stop['stop_lat'], stop['stop_lon']), axis=1)
    return stops.iloc[distances.idxmin()]

def get_times(start_lat, start_lon, end_lat, end_lon):
    # Find nearest stops to the start and end points
    start_stop = find_nearest_stop(start_lat, start_lon, stops)
    end_stop = find_nearest_stop(end_lat, end_lon, stops)
    
    # Calculate walking times to and from stops
    walk_to_start_time = calculate_distance(start_lat, start_lon, start_stop['stop_lat'], start_stop['stop_lon']) / WALKING_SPEED/60
    walk_from_end_time = calculate_distance(end_lat, end_lon, end_stop['stop_lat'], end_stop['stop_lon']) / WALKING_SPEED/60
    
    # Find trips that connect these stops
    start_trips = stop_times[stop_times['stop_id'] == start_stop['stop_id']]
    end_trips = stop_times[stop_times['stop_id'] == end_stop['stop_id']]
    connecting_trips = pd.merge(start_trips, end_trips, on='trip_id', suffixes=('_start', '_end'))
    
    # Filter for trips where the end stop comes after the start stop
    connecting_trips = connecting_trips[connecting_trips['stop_sequence_end'] > connecting_trips['stop_sequence_start']]
    
    # Calculate in-vehicle times
    connecting_trips['in_vehicle_time'] = (pd.to_datetime(connecting_trips['arrival_time_end']) - pd.to_datetime(connecting_trips['departure_time_start'])).dt.total_seconds()/60
    
    # Get the minimum in-vehicle time for the journey
    min_in_vehicle_time = connecting_trips['in_vehicle_time'].min() if not connecting_trips.empty else np.nan
    
    return walk_to_start_time, walk_from_end_time, min_in_vehicle_time




In [62]:
df0.columns

Index(['blockF', 'blockT', 'Trips_median', 'Trips_mean', 'Trips_sum', 'dist',
       'x_f', 'y_f', 'x_t', 'y_t', 'placeF', 'placeT', 'Lat_from', 'Lon_from',
       'Lat_to', 'Lon_to', 'network_node_id_ori', 'network_node_id_dest',
       'driving_dist', 'driving_time'],
      dtype='object')

In [63]:

# Calculate walking and in-vehicle times
results = df0.apply(lambda row: get_times(row['Lat_from'], row['Lon_from'], row['Lat_to'], row['Lon_to']), axis=1, result_type='expand')
results.columns = ['Walk_to_start_time', 'Walk_from_end_time', 'In_vehicle_time']

# Combine results with the original data
final_data = pd.concat([df0, results], axis=1)

C:\Users\behnamt\AppData\Local\Temp\ipykernel_19092\1863110363.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  connecting_trips['in_vehicle_time'] = (pd.to_datetime(connecting_trips['arrival_time_end']) - pd.to_datetime(connecting_trips['departure_time_start'])).dt.total_seconds()/60
C:\Users\behnamt\AppData\Local\Temp\ipykernel_19092\1863110363.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  connecting_trips['in_vehicle_time'] = (pd.to_datetime(connecting_trips['arrival_time_end']) - pd.to_datetime(connecting_trips['departure_time_start'])).dt.total_seconds()/60
C:\Users\behnamt\AppData\Local\Temp\ipykernel_19092\1863110363.py:42: UserWarning: Could not infer format, so each element will be parsed in

In [ ]:
final_data.columns

In [72]:
final_data['Walk_from_end_time'].max()

593.6457112482466

In [65]:
final_data.to_csv('trip_info.csv')

In [69]:
dft = final_data[~final_data['In_vehicle_time'].isna()]
dft['In_vehicle_time'].max()

50.7